In [1]:
%matplotlib inline

import sys
sys.path.append('../')

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('seaborn-white')
import pandas as pd
import seaborn as sns

import boolipy
api = boolipy.api.Api()

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


# Global variables

In [97]:
FRIDHEMSPLAN = [59.334438, 18.029522]
CENTER = ",".join([str(x) for x in FRIDHEMSPLAN])
DIM = "400,500"
LIMIT = 1000
ENDPOINT = "sold"

parameters = {"center": CENTER, "dim": DIM, "limit": LIMIT}

In [98]:
data = api.get(endpoint=ENDPOINT, parameters=parameters, follow=True)
df = pd.DataFrame(data[0][ENDPOINT])

DEBUG:boolipy.api:Get endpoint 'sold' with {'dim': '400,500', 'limit': 1000, 'center': '59.334438,18.029522'} and follow pagination: True
DEBUG:boolipy.api:Time from api 1566770669
DEBUG:boolipy.api:Get url: https://api.booli.se/sold with params {'dim': '400,500', 'limit': 1000, 'center': '59.334438,18.029522', 'callerId': 'aitorhh', 'time': '1566770669', 'hash': 'a5a677cf75c52906406dacee22e2eee54ec2665a', 'unique': 'CDKC1OXBIBB2MGTU'}
DEBUG:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): api.booli.se
DEBUG:requests.packages.urllib3.connectionpool:https://api.booli.se:443 "GET /sold?dim=400%2C500&limit=1000&center=59.334438%2C18.029522&callerId=aitorhh&time=1566770669&hash=a5a677cf75c52906406dacee22e2eee54ec2665a&unique=CDKC1OXBIBB2MGTU HTTP/1.1" 200 None
DEBUG:boolipy.api:{u'count': 985, u'searchParams': {u'dim': u'400,500', u'centerLng': u'18.029522', u'centerLat': u'59.334438'}, u'sold': [{u'booliId': 3544704, u'livingArea': 73, u'floor': 6, u'url': u'htt

DEBUG:boolipy.api:offset (0) + limit (1000) > total_count (985)


In [32]:
df

,additionalArea,apartmentNumber,booliId,constructionYear,floor,listPrice,livingArea,location,objectType,plotArea,published,rent,rooms,soldDate,soldPrice,soldPriceSource,source,url
0,0.0,1202,3235369,1903.0,2.0,2995000,37.0,"{u'position': {u'latitude': 59.33418961, u'lon...",Lägenhet,NaN,2018-09-08 10:30:28,1953,1.0,2018-09-19,3280000,bobot,"{u'url': u'http://www.erikolsson.se/', u'type'...",https://www.booli.se/annons/3235369
1,0.0,1001,3158877,1904.0,1.0,5995000,79.0,"{u'position': {u'latitude': 59.3347012, u'isAp...",Lägenhet,NaN,2018-05-18 21:35:16,2399,3.0,2018-06-05,6200000,bobot,"{u'url': u'http://www.erikolsson.se/', u'type'...",https://www.booli.se/annons/3158877
2,0.0,1501,3244761,1904.0,5.0,12900000,124.0,"{u'position': {u'latitude': 59.33468512, u'lon...",Lägenhet,NaN,2018-09-21 11:39:10,3766,4.0,2018-10-17,13000000,bobot,{u'url': u'https://www.svenskamaklarhuset.se/'...,https://www.booli.se/annons/3244761
3,0.0,1501,3323752,1982.0,5.0,6175000,84.0,"{u'position': {u'latitude': 59.33438477, u'lon...",Lägenhet,NaN,2019-03-10 01:46:30,3079,3.0,2019-03-29,6100000,bobot,{u'url': u'https://www.svenskamaklarhuset.se/'...,https://www.booli.se/annons/3323752
4,NaN,1302,3457400,1904.0,3.0,2900000,34.0,"{u'position': {u'latitude': 59.33422212, u'lon...",Lägenhet,NaN,2019-05-24 16:50:02,1770,1.0,2019-06-25,3075000,bid,"{u'url': u'http://www.alexanderwhite.se/', u't...",https://www.booli.se/annons/3457400
5,NaN,1002,3384692,1904.0,NaN,4150000,48.0,"{u'position': {u'latitude': 59.33449118, u'lon...",Lägenhet,NaN,2019-04-06 11:47:21,1530,2.0,2019-05-22,4390000,maklare,{u'url': u'http://www.visionfastighetsmakleri....,https://www.booli.se/annons/3384692
6,NaN,NaN,3462332,1982.0,2.0,5995000,84.0,"{u'position': {u'latitude': 59.33438477, u'lon...",Lägenhet,NaN,2019-05-29 17:55:49,3079,3.0,2019-06-14,6350000,bid,"{u'url': u'http://www.skandiamaklarna.se/', u'...",https://www.booli.se/annons/3462332
7,6.0,1502,2418901,1905.0,5.0,8500000,90.0,"{u'position': {u'latitude': 59.3347012, u'isAp...",Lägenhet,NaN,2017-10-11 10:46:08,2733,3.5,2017-11-01,9400000,maklare,"{u'url': u'http://www.erikolsson.se/', u'type'...",https://www.booli.se/annons/2418901
8,NaN,1201,3174477,1969.0,2.0,5795000,83.0,"{u'position': {u'latitude': 59.33474148, u'lon...",Lägenhet,NaN,2018-06-01 13:36:28,3096,3.0,2018-06-14,6075000,maklare,{u'url': u'https://nestorfastighetsmakleri.se/...,https://www.booli.se/annons/3174477
9,0.0,NaN,3440177,1904.0,2.0,3995000,46.0,"{u'position': {u'latitude': 59.33468886, u'lon...",Lägenhet,NaN,2019-05-09 21:07:31,1467,2.0,2019-05-29,4400000,bid,"{u'url': u'http://www.husmanhagberg.se/', u'ty...",https://www.booli.se/annons/3440177


{u'address': {u'streetAddress': u'Fridhemsgatan 29C'},
 u'distance': {u'ocean': 2379},
 u'namedAreas': [u'Kungsholmen'],
 u'position': {u'latitude': 59.33418961, u'longitude': 18.02980661},
 u'region': {u'countyName': u'Stockholms l\xe4n',
  u'municipalityName': u'Stockholm'}}

In [101]:
from haversine import haversine, Unit

# define how do we want to change the data
def time_published(x): return x["soldDate"]-x["published"]
def source_name(x): return x["source"]["name"]
def price_increased(x): return x["soldPrice"] - x["listPrice"]
# convert location to tuple
def location_tuple(x): 
    loc = x['location']['position']
    return (loc['latitude'], loc['longitude'])
def dist_center(x, center = CENTER): 
    cloc = center
    if isinstance(center, str):
        cloc = tuple([float(l) for l in center.split(',')])

    # using default location
    #loc = x['location']['position']
    #c = (loc['latitude'], loc['longitude'])
    
    # using new loc_tuple
    c = x['loc_tuple']

    return haversine(c, cloc, unit=Unit.METERS)

# test distance
#d = {"location": {"position": {"latitude": 59.334438, "longitude": 18.029522}}}
#center = CENTER
#assert dist_center(d, center) == 0

In [102]:
# apply all transformations
###########################
# transform data
df['loc_tuple'] = df.apply(location_tuple, axis=1)
df['soldDate'] = pd.to_datetime(df['soldDate'], format="%Y-%m-%d") 
df['published'] = pd.to_datetime(df['published']) 


# add new data
df['time_published'] = df.apply(time_published, axis=1)
df['source_name'] = df.apply(source_name, axis=1)
df['price_increased'] = df.apply(price_increased, axis=1)

In [103]:
df

,additionalArea,apartmentNumber,booliId,constructionYear,floor,isNewConstruction,listPrice,livingArea,location,objectType,...,rooms,soldDate,soldPrice,soldPriceSource,source,url,loc_tuple,time_published,source_name,price_increased
0,10.0,1601,3544704,1928.0,6.0,NaN,6495000.0,73.0,"{u'position': {u'latitude': 59.33530608, u'lon...",Lägenhet,...,2.5,2019-08-22,7100000,maklare,{u'url': u'http://bronzefastighetsformedling.s...,https://www.booli.se/annons/3544704,"(59.33530608, 18.02698387)",7 days 07:12:01,Bronze Fastighetsförmedling AB,605000.0
1,NaN,NaN,3479102,1941.0,5.0,NaN,2995000.0,33.0,"{u'position': {u'latitude': 59.33546578, u'lon...",Lägenhet,...,1.0,2019-08-22,3100000,bobot,"{u'url': u'http://www.mohv.se/', u'type': u'Br...",https://www.booli.se/annons/3479102,"(59.33546578, 18.02978754)",70 days 07:16:00,MOHV,105000.0
2,0.0,NaN,3537999,1903.0,2.0,NaN,4995000.0,59.8,"{u'position': {u'latitude': 59.33623, u'longit...",Lägenhet,...,2.0,2019-08-21,5525000,bobot,"{u'url': u'http://www.notar.se/', u'type': u'B...",https://www.booli.se/annons/3537999,"(59.33623, 18.032)",13 days 12:19:25,Notar,530000.0
3,NaN,NaN,3540936,1905.0,0.5,NaN,6375000.0,84.0,"{u'position': {u'latitude': 59.33336836, u'lon...",Lägenhet,...,3.0,2019-08-21,6575000,maklare,"{u'url': u'http://www.maklarhuset.se/', u'type...",https://www.booli.se/annons/3540936,"(59.33336836, 18.0338338)",11 days 05:05:19,Mäklarhuset,200000.0
4,0.0,1405,3245178,1942.0,4.0,NaN,2495000.0,26.0,"{u'position': {u'latitude': 59.33320823, u'lon...",Lägenhet,...,1.0,2019-08-20,2725000,maklare,"{u'url': u'http://www.skandiamaklarna.se/', u'...",https://www.booli.se/annons/3245178,"(59.33320823, 18.03295279)",17 days 06:18:46,SkandiaMäklarna,230000.0
5,0.0,NaN,3476073,1942.0,1.0,NaN,2995000.0,34.0,"{u'position': {u'latitude': 59.33320823, u'lon...",Lägenhet,...,2.0,2019-08-15,3450000,bobot,"{u'url': u'http://www.notar.se/', u'type': u'B...",https://www.booli.se/annons/3476073,"(59.33320823, 18.03295279)",12 days 05:32:28,Notar,455000.0
6,0.0,NaN,3519427,1905.0,NaN,NaN,2995000.0,35.0,"{u'position': {u'latitude': 59.33336836, u'lon...",Lägenhet,...,2.0,2019-08-15,3500000,bid,"{u'url': u'http://www.skandiamaklarna.se/', u'...",https://www.booli.se/annons/3519427,"(59.33336836, 18.0338338)",11 days 23:57:30,SkandiaMäklarna,505000.0
7,0.0,NaN,3485772,1903.0,4.0,NaN,4995000.0,59.8,"{u'position': {u'latitude': 59.33623, u'longit...",Lägenhet,...,2.0,2019-08-14,5635000,bid,"{u'url': u'http://www.husmanhagberg.se/', u'ty...",https://www.booli.se/annons/3485772,"(59.33623, 18.032)",11 days 05:32:03,HusmanHagberg,640000.0
8,1.0,1502,3014012,1890.0,5.0,NaN,6750000.0,79.0,"{u'position': {u'latitude': 59.33500192, u'lon...",Lägenhet,...,3.0,2019-01-16,6750000,maklare,"{u'url': u'http://www.skandiamaklarna.se/', u'...",https://www.booli.se/annons/3014012,"(59.33500192, 18.02963026)",11 days 05:45:30,SkandiaMäklarna,0.0
9,0.0,1204,3524296,1942.0,2.0,NaN,3250000.0,34.0,"{u'position': {u'latitude': 59.33327343, u'lon...",Lägenhet,...,2.0,2019-08-07,3000000,bid,"{u'url': u'http://www.innerspec.se/', u'type':...",https://www.booli.se/annons/3524296,"(59.33327343, 18.03275961)",25 days 03:15:55,Innerstadsspecialisten AB,-250000.0


# Plot apartments

In [106]:
import folium

def get_popup_str(x): return 'size: {} </br> price: {} </br> date: {}'.format(x['livingArea'], x['soldPrice'], x['soldDate'])

m = folium.Map(
    location=FRIDHEMSPLAN,
    zoom_start=15
)

# center marker
folium.Marker(FRIDHEMSPLAN, popup='Center of the univers').add_to(m)

# apartments found
for index, row in df.iterrows():
    folium.Marker(row['loc_tuple'], popup=get_popup_str(row)).add_to(m)

m